In [1]:
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV

from sign.CONST import AMOUNT_OF_KEYFRAMES

from sign.training.load_data.dynamic_loader import DynamicLoader

CSV_PATH = 'dyn_50_common_youtube_55p.csv'
NUM_HANDS = 2

dyna_lod = DynamicLoader(AMOUNT_OF_KEYFRAMES)
training_data = dyna_lod.prepare_training_data(CSV_PATH)
xs,ys,xs_test,ys_test = training_data.spliterino(random_state=1)


🔥🔥 TrajectoryBuilder is now running in BERTRAM_MODE 🔥🔥


# LogisticRegression

In [13]:

from sklearn.linear_model import LogisticRegression

logistic_reg = LogisticRegression(random_state=42,max_iter=10000,C=90)
parameters = {'C':range(1000,10000,200)}
pre_clf = RandomizedSearchCV(logistic_reg,parameters,n_jobs=-1)
pre_clf.fit(np.array(xs),ys)
print(pre_clf.best_estimator_,pre_clf.best_index_,pre_clf.best_params_,pre_clf.best_score_)
print(classification_report(ys_test,logistic_reg.predict(np.array(xs_test))))


# Lav grid search hmm maybe?

(LogisticRegression(C=930, max_iter=10000, random_state=42),
 6,
 {'C': 930},
 0.5352519009026157)

# SVC / SVM

In [3]:

from sklearn.svm import SVC
svc = SVC(random_state=42)
parameters = {'C':[float(x) for x in range(0,2000,200)],"gamma":[x * 0.1 for x in range(0,51,1)]}
pre_clf = RandomizedSearchCV(svc,parameters,n_jobs=-1)
pre_clf.fit(np.array(xs),ys)
print(pre_clf.best_estimator_,pre_clf.best_index_,pre_clf.best_params_,pre_clf.best_score_)
print(classification_report(ys_test,svc.predict(np.array(xs_test))))



(SVC(C=400.0, gamma=0.4, random_state=42),
 0,
 {'gamma': 0.4, 'C': 400.0},
 0.5751985724095497)

# Random Forest Classifier

In [ ]:

from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()
parameters = {"max_features":["sqrt","log2"],"n_estimators":range(100,1001,100)}
pre_clf = RandomizedSearchCV(random_forest,parameters)
pre_clf.fit(np.array(xs),ys)
print(pre_clf.best_estimator_,pre_clf.best_index_,pre_clf.best_params_,pre_clf.best_score_)
print(classification_report(ys_test,random_forest.predict(np.array(xs_test))))



# Voting Classifier

In [3]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

voting_clf = VotingClassifier( estimators=[ 
                                           ('lr', LogisticRegression(C=930.0,max_iter=10_000)), 
                                           ('rf', RandomForestClassifier(random_state=42,n_estimators=500,max_features="sqrt")), 
                                           ('svc', SVC(random_state=42,probability=True,gamma=0.4,C=400.0))
]) 
parameters = {"voting":["hard","soft"]}
pre_clf = RandomizedSearchCV(voting_clf,parameters,n_jobs=2)
pre_clf.fit(np.array(xs),ys)
print("VotingClassifier")
print(pre_clf.best_estimator_,pre_clf.best_index_,pre_clf.best_params_,pre_clf.best_score_)
print(classification_report(ys_test,voting_clf.predict(xs_test)))


/mnt/repositories/Bsc-Sign-Language-Recon/backend/.venv/lib/python3.11/site-packages/sklearn/model_selection/_search.py:318: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


# Bagging Classifier

In [ ]:
from sklearn.ensemble import BaggingClassifier

bagging_clf = BaggingClassifier(estimator=[ 
                                           ('lr', LogisticRegression(C=930.0,max_iter=10_000)), 
                                           ('rf', RandomForestClassifier(random_state=42,n_estimators=500,max_features="sqrt")), 
                                           ('svc', SVC(random_state=42,probability=True,gamma=0.4,C=400.0))
]) 
bagging_clf.fit(xs, ys)

parameters = {"voting":["hard","soft"]}
pre_clf = RandomizedSearchCV(bagging_clf,parameters,n_jobs=-1)
pre_clf.fit(np.array(xs),ys)
print("VotingClassifier")
print(pre_clf.best_estimator_,pre_clf.best_index_,pre_clf.best_params_,pre_clf.best_score_)
print(classification_report(ys_test,bagging_clf.predict(xs_test)))
